In [1]:
import numpy as np
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, Aer
from qiskit.visualization import plot_histogram, plot_state_qsphere

# Teleportación cuántica 
En este taller veremos una de las aplicaciones más sorprendentes, contraintuitivas y divertidas de la mecánica cuántica, la **teleportación cuántica**. Esta nos permite mover estados cuánticos entre dos puntos sin la necesidad de un canal de comunicación físico, sino que solo a través de un estado entrelazado.

Consideremos dos personas, Alice y Bob, los cuales se encontraron hace mucho tiempo, pero que ahora viven muy separados. Cuando estuvieron juntos ellos generaron el siguiente estado entrelazado
$$ |EPR\rangle = \frac{1}{\sqrt{2}}( |0_A0_B\rangle + |1_A1_B\rangle ) .$$
Al separarse cada uno se llevó consigo uno de los qubits de este estado. En la actualidad, Alice tiene la misión de enviarle a Bob un estado cuántico $|\psi\rangle$ que este no conoce, pero solamente pueden intercambiar información clásica a través de un teléfono. 

Esta no es una tarea sencilla para Alice. Dado que no conoce el estado $|\psi\rangle$, no puede simplemente llamar a Bob y decírselo. Incluso si conociera el estado $|\psi\rangle$, este es un vector definido en un espacio continuo, por lo que Alice requeriría una cantidad infinita de tiempo para poder comunicárselo a Bob. 

¿Existe alguna otra alternativa? Claro que sí! Alice puede usar el estado $|EPR\rangle$ que compartió con Bob en el pasado y realizar una teleportación cuántica! Ahora implementaremos paso por paso este protocolo con Qiskit.

**Ejercicio 1: Estados iniciales.**

Cree un circuito cuántico llamado `qc_in` de 3 qubits, dos para Alice y uno para Bob, y 2 bits clásicos. Posteriormente, prepare en el primer qubit de Alice el estado cuántico 
$$|\psi\rangle = \frac{1}{\sqrt{2}}( |0_A\rangle + i|1_A\rangle )$$
y entre el segundo qubit de Alice y el de Bob el estado entrelazado $|EPR\rangle$. De este modo, el estado inicial del circuito es
$$ |\Psi_{in}\rangle = |\psi\rangle \otimes |EPR\rangle = \frac{1}{2}( |0_A0_A0_B\rangle +|0_A1_A1_B\rangle +i|1_A0_A0_B\rangle +i|1_A1_A1_B\rangle      ).$$

In [ ]:
### solucion
qba = QuantumRegister( , name='alice' )
qbb = QuantumRegister( , name='bob' )
cb  = ClassicalRegister( , name='clbit' )

qc_in = QuantumCircuit( qba, qbb, cb )

#Alice


#EPR (Alice y Bob)


qc_in.draw( output='mpl' )

**Ejercicio 2: Medida de Bell.**

Cree otro circuito de 3 qubits y 2 clbits llamado `qc_bell`. Implemente una medida de Bell entre los dos primeros qubits. Esta es una medida entrelazada que se implementa aplicando una $CX$ entre los dos qubit, una $H$ en el primero qubit, y mediciones en ambos qubits.

In [ ]:
qc_bell = QuantumCircuit( qba, qbb, cb )
#####
#solucion, implementar gates de medida

#####
qc_bell.measure( qba, cb )
qc_bell.draw( output='mpl' )

**Ejercicio 3: Composición de circuitos.**

Cree un nuevo circuito llamado `qc` de 3 qubits y 2 clbits. Componga los circuitos `qc`, `qc_in` y `qc_bell` con la función `compose`. El estado de este circuito justo antes de la medida es
\begin{align*}
|\Psi_{out}\rangle =& \frac{1}{2\sqrt{2}}\big[ |0_A0_A\rangle(|0_B\rangle+i|1_B\rangle) + |0_A1_A\rangle(|1_B\rangle+i|0_B\rangle) \\
& \qquad + |1_A0_A\rangle(|0_B\rangle-i|1_B\rangle) + |1_A1_A\rangle(|1_B\rangle-i|0_B\rangle) \big].
\end{align*}


**Problema 4: Puertas condicionadas clásicas.**

Del estado $|\Psi_{out}\rangle$ podemos ver el estado de Bob una vez que Alice realiza una medida sobre sus dos qubits.

\begin{equation*}
\begin{matrix}
00 & \rightarrow &|0\rangle + i|1\rangle\\
01 & \rightarrow &|1\rangle + i|0\rangle\\
10 & \rightarrow &|0\rangle - i|1\rangle\\
11 & \rightarrow &|1\rangle + i|0\rangle
\end{matrix}
\end{equation*}

De este modo, si obtenemos el resultado $00$ el estado de Bob es exactamente el estado $|\psi\rangle$. Sin embargo, cualquiera de los otros resultados nos da un estado diferente. ¿Puede Bob arreglar sus estados en ese caso? La respuesta es que sí, pero requiere que Alice le comunique por teléfono (clásicamente) el resultado de sus medidas:

* Si el resultado fue $00$, Bob no hace nada.
* Si el resultado fue $01$, Bob puede arreglar su estado aplicando un puerta $X$.
* Si el resultado fue $10$, Bob puede arreglar su estado aplicando un puerta $Z$.
* Si el resultado fue $11$, Bob puede arreglar su estado aplicando primero una puerta $X$ y después un puerta $Z$.
 
Todo esto puede resumirse en que Bob aplica la puerta $Z^{m_1}X^{m_0}$, la cual esta controlada clásicamente por los resultados de las medidas $m_2m_1$ de Alice. Esta operación puede ser implementada fácilmente mediante la función `c_if`. 

Aguegue la operación clásicamente controlada $Z^{m_1}X^{m_0}$ al circuito `qc`.

**Ejercicio 5: Simulación** 

Ejecute el circuito `qc` con el `statevector_simulator` y verifique que el estado del circuito es igual al estado $|\psi\rangle$. Note que el resultado de la simualción será un estado de 3 qubits, de modo que debemos compararlo con 
\begin{equation*}
|\psi\rangle\otimes|0\rangle\otimes|0\rangle = \frac{1}{\sqrt{2}}( |000\rangle + i|100\rangle ).
\end{equation*}

In [3]:
import qiskit.tools.jupyter
%qiskit_version_table